In [8]:
!git clone https://github.com/Only-Mike/M2-NLP-Network-Analysis.git

Cloning into 'M2-NLP-Network-Analysis'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [2]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 1.7 MB 4.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [3]:
# explainability (why did the model say it's related to this author)
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.3 MB/s 
     |████████████████████████████████| 133 kB 60.8 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=d83193053fa36f9202b9fda56a795a32a32fde833453895e76ec1d9867db45ba
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
import preprocessor as prepro # text prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [10]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [9]:
data1 = pd.read_csv('/content/M2-NLP-Network-Analysis/Supply chain network - 2017 csv file.csv')
data2 = pd.read_csv('/content/M2-NLP-Network-Analysis/Supply chain network - 2018 csv file.csv')
data3 = pd.read_csv('/content/M2-NLP-Network-Analysis/Supply chain network - 2019 csv file.csv')
data4 = pd.read_csv('/content/M2-NLP-Network-Analysis/Supply chain network - 2020 csv file.csv')
data5 = pd.read_csv('/content/M2-NLP-Network-Analysis/Supply chain network - 2021 csv file.csv')

In [12]:
#framing and merging the data
frames = [data1, data2, data3, data4, data5]
df = pd.concat(frames)

In [13]:
df

,Authors,Author(s) ID,Title,Year,Source title,Cited by,DOI,Link,Affiliations,Authors with affiliations,...,Index Keywords,Funding Details,Correspondence Address,Publisher,ISSN,ISBN,CODEN,Document Type,Source,EID
0,"Crush J., Frayne B., McCordic C.",7003748511;6507551740;56004012500;,Food and nutrition security in southern Africa...,2017,Food and Nutrition Security in Southern Africa...,4.0,10.4324/9781315226651,https://www.scopus.com/inward/record.uri?eid=2...,"University of Cape Town, South Africa; School ...","Crush, J., University of Cape Town, South Afri...",...,NaN,NaN,"Crush, J.; University of Cape TownSouth Africa",Taylor and Francis,NaN,9781315226651; 9780415786782,NaN,Book,Scopus,2-s2.0-85046049829
1,"Tsang Y.P., Choy K.L., Wu C.H., Ho G.T.S., Lam...",57191666025;7005477047;55506191100;8348673600;...,An IoT-based cargo monitoring system for enhan...,2017,International Journal of Engineering Business ...,25.0,10.1177/1847979017749063,https://www.scopus.com/inward/record.uri?eid=2...,Department of Industrial and Systems Engineeri...,"Tsang, Y.P., Department of Industrial and Syst...",...,NaN,PolyU,"Tsang, Y.P.; Department of Industrial and Syst...",SAGE Publications Inc.,18479790,NaN,NaN,Article,Scopus,2-s2.0-85039850983
2,[No author name available],[No author id available],2017 IEEE 28th Annual Software Technology Conf...,2017,2017 IEEE 28th Annual Software Technology Conf...,NaN,NaN,https://www.scopus.com/inward/record.uri?eid=2...,NaN,NaN,...,NaN,NaN,NaN,Institute of Electrical and Electronics Engine...,NaN,9781538610886,NaN,Conference Review,Scopus,2-s2.0-85050200429
3,Secomandi N.,6506411578;,Approximations for high dimensional commodity ...,2017,"Foundations and Trends in Technology, Informat...",3.0,10.1561/0200000080,https://www.scopus.com/inward/record.uri?eid=2...,"Tepper School of Business, Carnegie Mellon Uni...","Secomandi, N., Tepper School of Business, Carn...",...,Electric energy storage; Energy conversion; Ri...,NaN,"Secomandi, N.; Tepper School of Business, Unit...",Now Publishers Inc,15719545,NaN,NaN,Conference Paper,Scopus,2-s2.0-85039788219
4,"Korhonen K., Kotavaara O., Muilu T., Rusanen J.",57200382781;35772315700;6602393288;6601955741;,Accessibility of Local Food Production to Regi...,2017,European Countryside,10.0,10.1515/euco-2017-0040,https://www.scopus.com/inward/record.uri?eid=2...,"Natural Resources Institute Finland, Universit...","Korhonen, K., Natural Resources Institute Finl...",...,accessibility; agricultural economics; food ma...,NaN,NaN,Walter de Gruyter GmbH,18038417,NaN,NaN,Article,Scopus,2-s2.0-85041059781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Gawade D.,57224469235;,Agile Supply Chain in Manufacturing and Servic...,2021,Vision,1.0,10.1177/09722629211002000,https://www.scopus.com/inward/record.uri?eid=2...,"Symbiosis Institute of Operations Management, ...","Gawade, D., Symbiosis Institute of Operations ...",...,NaN,NaN,"Gawade, D.; Symbiosis Institute of Operations ...",Sage Publications India Pvt. Ltd,09722629,NaN,NaN,Review,Scopus,2-s2.0-85107617698
1996,"Huang L., Tan Y., Guan X.",55794665100;57223669545;54385196800;,Evaluation of Cruise Ship Supply Logistics Ser...,2021,Journal of Advanced Transportation,1.0,10.1155/2021/6645946,https://www.scopus.com/inward/record.uri?eid=2...,"School of Management, Wuhan Polytechnic Univer...","Huang, L., School of Management, Wuhan Polytec...",...,Internet service providers; Radial basis funct...,NaN,"Huang, L.; School of Management, China; email:...",Hindawi Limited,01976729,NaN,JATRD,Article,Scopus,2-s2.0-85107579417
1997,"Agrawal R., Wankhede V.A., Kumar A., Luthra S.",57193951175;57204053781;57001679600;43361407000;,A systematic and network-based analysis of dat...,2021,TQM Journal,4.0,10.1108/TQM-12-2020-0285,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Production Engineering, National...","Agrawal, R., Department of Production Engineer...",...,Artificial intelli